In [1]:
# Import Dependancies
import pandas as pd
from sqlalchemy import create_engine
from Postgres_Login import username, password

In [2]:
# Connect to Database
rds_connection_string = f"{username}:{password}@localhost:5432/streaming_services"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [3]:
# Hulu - Reading CSV
filePath = "Resources/HuluRaw_AP.csv"
data = pd.read_csv(filePath, encoding='utf-8')

In [4]:
# Hulu - Storing Selected data from CSV into DataFrame
dataCleaned = data[['show/name','show/genre','show/seasons_count',
                    'show/show_rollups/showtime/highest_rating']]
dataCleaned.head()

,show/name,show/genre,show/seasons_count,show/show_rollups/showtime/highest_rating
0,Family Guy,Animation and Cartoons,15,TV-MA
1,South Park,Comedy,20,TV-MA
2,American Dad!,Comedy,9,TV-14
3,Law & Order: Special Victims Unit,Drama,18,TV-14
4,Naruto Shippuden,Anime,9,TV-14


In [5]:
# Hulu - Renaming Columns to Match Other Tables and Getting Rid of NULL variables. Adding New Column to specify it can be found in Hulu.
dataCleaned = dataCleaned.rename(columns={'show/name':'Title of Show',
                                          'show/genre':'Genres',
                                          'show/seasons_count':'No. of Seasons',
                                          'show/show_rollups/showtime/highest_rating':'TV Rating'})
dataCleaned['Hulu'] = 'Yes'
dataCleaned['TV Rating'].fillna("Not Rated", inplace = True)
dataCleaned

,Title of Show,Genres,No. of Seasons,TV Rating,Hulu
0,Family Guy,Animation and Cartoons,15,TV-MA,Yes
1,South Park,Comedy,20,TV-MA,Yes
2,American Dad!,Comedy,9,TV-14,Yes
3,Law & Order: Special Victims Unit,Drama,18,TV-14,Yes
4,Naruto Shippuden,Anime,9,TV-14,Yes
...,...,...,...,...,...
995,East Los High,Drama,4,TV-14,Yes
996,Sesame Street,Kids,3,TV-Y,Yes
997,Casual,Comedy,3,TV-MA,Yes
998,Chicago P.D.,Drama,1,TV-14,Yes


In [6]:
# Hulu - Sending to PostgresSQL
dataCleaned.to_sql(name='hulu', con=engine, if_exists='append', index=False)
pd.read_sql_query('select * from hulu', con=engine)


,Title of Show,Genres,No. of Seasons,TV Rating,Hulu
0,Family Guy,Animation and Cartoons,15,TV-MA,Yes
1,South Park,Comedy,20,TV-MA,Yes
2,American Dad!,Comedy,9,TV-14,Yes
3,Law & Order: Special Victims Unit,Drama,18,TV-14,Yes
4,Naruto Shippuden,Anime,9,TV-14,Yes
...,...,...,...,...,...
995,East Los High,Drama,4,TV-14,Yes
996,Sesame Street,Kids,3,TV-Y,Yes
997,Casual,Comedy,3,TV-MA,Yes
998,Chicago P.D.,Drama,1,TV-14,Yes


In [7]:
# Netflix - Reading and Storing Selected data from CSV into DataFrame
csv_file = "Resources/netflix_dataset_EE.csv"
netflix_data_df = pd.read_csv(csv_file)
netflix_data_df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...
...,...,...,...,...,...,...,...,...,...,...,...,...
6229,80000063,TV Show,Red vs. Blue,NaN,"Burnie Burns, Jason Saldaña, Gustavo Sorola, G...",United States,NaN,2015,NR,13 Seasons,"TV Action & Adventure, TV Comedies, TV Sci-Fi ...","This parody of first-person shooter games, mil..."
6230,70286564,TV Show,Maron,NaN,"Marc Maron, Judd Hirsch, Josh Brener, Nora Zeh...",United States,NaN,2016,TV-MA,4 Seasons,TV Comedies,"Marc Maron stars as Marc Maron, who interviews..."
6231,80116008,Movie,Little Baby Bum: Nursery Rhyme Friends,NaN,NaN,NaN,NaN,2016,NaN,60 min,Movies,Nursery rhymes and original music for children...
6232,70281022,TV Show,A Young Doctor's Notebook and Other Stories,NaN,"Daniel Radcliffe, Jon Hamm, Adam Godley, Chris...",United Kingdom,NaN,2013,TV-MA,2 Seasons,"British TV Shows, TV Comedies, TV Dramas","Set during the Russian Revolution, this comic ..."


In [8]:
# Netflix - Dropping all Movies to keep only TV Shows
new_netflix_data_df = netflix_data_df.loc[netflix_data_df['type'] == 'TV Show']
new_netflix_data_df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
5,80163890,TV Show,Apaches,NaN,"Alberto Ammann, Eloy Azorín, Verónica Echegui,...",Spain,"September 8, 2017",2016,TV-MA,1 Season,"Crime TV Shows, International TV Shows, Spanis...",A young journalist is forced into a life of cr...
8,80117902,TV Show,Fire Chasers,NaN,NaN,United States,"September 8, 2017",2017,TV-MA,1 Season,"Docuseries, Science & Nature TV","As California's 2016 fire season rages, brave ..."
26,80244601,TV Show,Castle of Stars,NaN,"Chaiyapol Pupart, Jintanutda Lummakanon, Worra...",NaN,"September 7, 2018",2015,TV-14,1 Season,"International TV Shows, Romantic TV Shows, TV ...",As four couples with different lifestyles go t...
...,...,...,...,...,...,...,...,...,...,...,...,...
6228,80159925,TV Show,Kikoriki,NaN,Igor Dmitriev,NaN,NaN,2010,TV-Y,2 Seasons,Kids' TV,A wacky rabbit and his gang of animal pals hav...
6229,80000063,TV Show,Red vs. Blue,NaN,"Burnie Burns, Jason Saldaña, Gustavo Sorola, G...",United States,NaN,2015,NR,13 Seasons,"TV Action & Adventure, TV Comedies, TV Sci-Fi ...","This parody of first-person shooter games, mil..."
6230,70286564,TV Show,Maron,NaN,"Marc Maron, Judd Hirsch, Josh Brener, Nora Zeh...",United States,NaN,2016,TV-MA,4 Seasons,TV Comedies,"Marc Maron stars as Marc Maron, who interviews..."
6232,70281022,TV Show,A Young Doctor's Notebook and Other Stories,NaN,"Daniel Radcliffe, Jon Hamm, Adam Godley, Chris...",United Kingdom,NaN,2013,TV-MA,2 Seasons,"British TV Shows, TV Comedies, TV Dramas","Set during the Russian Revolution, this comic ..."


In [9]:
# Netflix - Renaming Columns to Match Other Tables and Getting Rid of NULL variables. Adding New Column to specify it can be found in Netflix.
clean_netflix_data_df = new_netflix_data_df[['title', 'listed_in', 'duration', 'rating']]
clean_netflix_data = clean_netflix_data_df.rename(columns = {"title" : "Title of Show",
                                                          "listed_in" : "Genres",
                                                          "duration" : "No. of Seasons",
                                                          "rating" : "TV Rating"})
final_netflix_data = clean_netflix_data.dropna()
final_netflix_data['Netflix'] = 'Yes'
final_netflix_data

,Title of Show,Genres,No. of Seasons,TV Rating,Netflix
2,Transformers Prime,Kids' TV,1 Season,TV-Y7-FV,Yes
3,Transformers: Robots in Disguise,Kids' TV,1 Season,TV-Y7,Yes
5,Apaches,"Crime TV Shows, International TV Shows, Spanis...",1 Season,TV-MA,Yes
8,Fire Chasers,"Docuseries, Science & Nature TV",1 Season,TV-MA,Yes
26,Castle of Stars,"International TV Shows, Romantic TV Shows, TV ...",1 Season,TV-14,Yes
...,...,...,...,...,...
6228,Kikoriki,Kids' TV,2 Seasons,TV-Y,Yes
6229,Red vs. Blue,"TV Action & Adventure, TV Comedies, TV Sci-Fi ...",13 Seasons,NR,Yes
6230,Maron,TV Comedies,4 Seasons,TV-MA,Yes
6232,A Young Doctor's Notebook and Other Stories,"British TV Shows, TV Comedies, TV Dramas",2 Seasons,TV-MA,Yes


In [10]:
# Netflix - Removing seasons from No. of Seasons to match others
final_netflix_data["No. of Seasons"] = final_netflix_data["No. of Seasons"].str.strip("Seasons").astype(str)
final_netflix_data

,Title of Show,Genres,No. of Seasons,TV Rating,Netflix
2,Transformers Prime,Kids' TV,1,TV-Y7-FV,Yes
3,Transformers: Robots in Disguise,Kids' TV,1,TV-Y7,Yes
5,Apaches,"Crime TV Shows, International TV Shows, Spanis...",1,TV-MA,Yes
8,Fire Chasers,"Docuseries, Science & Nature TV",1,TV-MA,Yes
26,Castle of Stars,"International TV Shows, Romantic TV Shows, TV ...",1,TV-14,Yes
...,...,...,...,...,...
6228,Kikoriki,Kids' TV,2,TV-Y,Yes
6229,Red vs. Blue,"TV Action & Adventure, TV Comedies, TV Sci-Fi ...",13,NR,Yes
6230,Maron,TV Comedies,4,TV-MA,Yes
6232,A Young Doctor's Notebook and Other Stories,"British TV Shows, TV Comedies, TV Dramas",2,TV-MA,Yes


In [11]:
# Netflix - Sending to PostgresSQL
final_netflix_data.to_sql(name='netflix', con=engine, if_exists='append', index=False)
pd.read_sql_query('select * from netflix', con=engine).head()


,Title of Show,Genres,No. of Seasons,TV Rating,Netflix
0,Transformers Prime,Kids' TV,1,TV-Y7-FV,Yes
1,Transformers: Robots in Disguise,Kids' TV,1,TV-Y7,Yes
2,Apaches,"Crime TV Shows, International TV Shows, Spanis...",1,TV-MA,Yes
3,Fire Chasers,"Docuseries, Science & Nature TV",1,TV-MA,Yes
4,Castle of Stars,"International TV Shows, Romantic TV Shows, TV ...",1,TV-14,Yes


In [12]:
# Prime - Reading and Storing Selected data from CSV into DataFrame
csv_file = "Resources/Prime_TV_Shows_Dataset_JC.csv"
Raw_Prime_Data = pd.read_csv(csv_file)

In [13]:
# Prime - Renaming Columns to Match Other Tables and Getting Rid of NULL variables. Adding New Column to specify it can be found in Prime.
Focused_Prime_Data = Raw_Prime_Data[["Name of the show", "No of seasons available", "Genre", "Age of viewers"]]

Cleaned_Prime_Data = Focused_Prime_Data.rename(columns = {"Name of the show" : "Title of Show",
                                                          "No of seasons available" : "No. of Seasons",
                                                          "Genre" : "Genres",
                                                          "Age of viewers" : "TV Rating"})

Cleaned_Prime_Data = Cleaned_Prime_Data [["Title of Show", "Genres", "No. of Seasons", "TV Rating"]]

Cleaned_Prime_Data["Prime"] = "Yes"

Cleaned_Prime_Data = Cleaned_Prime_Data.dropna()

Cleaned_Prime_Data

,Title of Show,Genres,No. of Seasons,TV Rating,Prime
0,Pataal Lok,Drama,1.0,18+,Yes
1,Upload,Sci-fi comedy,1.0,16+,Yes
2,The Marvelous Mrs. Maisel,"Drama, Comedy",3.0,16+,Yes
3,Four More Shots Please,"Drama, Comedy",2.0,18+,Yes
4,Fleabag,Comedy,2.0,18+,Yes
...,...,...,...,...,...
388,Chris Tall Presents�,Comedy,1.0,16+,Yes
389,Short Poppies,Comedy,1.0,13+,Yes
390,The 2018 Rose Parade Hosted by Cord & Tish,Comedy,1.0,All,Yes
391,Aliens Love Underpants And�,Kids,1.0,All,Yes


In [14]:
# Prime - Removing "+" and Renaming Ages to Match Offical TV Rating Titles.
Cleaned_Prime_Data["TV Rating"] = Cleaned_Prime_Data["TV Rating"].str.strip("+").astype(str)
Cleaned_Prime_Data["TV Rating"].replace(regex = True, inplace = True, to_replace = "18", value = "TV-MA")
Cleaned_Prime_Data["TV Rating"].replace(regex = True, inplace = True, to_replace = "16", value = "TV-MA")
Cleaned_Prime_Data["TV Rating"].replace(regex = True, inplace = True, to_replace = "13", value = "TV-14")
Cleaned_Prime_Data["TV Rating"].replace(regex = True, inplace = True, to_replace = "7", value = "TV-7")
Cleaned_Prime_Data["TV Rating"].replace(regex = True, inplace = True, to_replace = "All", value = "TV-G")
Cleaned_Prime_Data

,Title of Show,Genres,No. of Seasons,TV Rating,Prime
0,Pataal Lok,Drama,1.0,TV-MA,Yes
1,Upload,Sci-fi comedy,1.0,TV-MA,Yes
2,The Marvelous Mrs. Maisel,"Drama, Comedy",3.0,TV-MA,Yes
3,Four More Shots Please,"Drama, Comedy",2.0,TV-MA,Yes
4,Fleabag,Comedy,2.0,TV-MA,Yes
...,...,...,...,...,...
388,Chris Tall Presents�,Comedy,1.0,TV-MA,Yes
389,Short Poppies,Comedy,1.0,TV-14,Yes
390,The 2018 Rose Parade Hosted by Cord & Tish,Comedy,1.0,TV-G,Yes
391,Aliens Love Underpants And�,Kids,1.0,TV-G,Yes


In [15]:
# Prime - Created New CSV of Cleaned Data
Cleaned_Prime_Data.to_csv("Resources/Cleaned_Prime_TV_Shows_Dataset_JC.csv")

In [16]:
# Prime - Sending to PostgresSQL
Cleaned_Prime_Data.to_sql(name = "prime", con = engine, if_exists = "append", index = False)

In [17]:
# Prime - Table Preview from database
pd.read_sql_query("select * from prime", con = engine).head()

,Title of Show,Genres,No. of Seasons,TV Rating,Prime
0,Pataal Lok,Drama,1.0,TV-MA,Yes
1,Upload,Sci-fi comedy,1.0,TV-MA,Yes
2,The Marvelous Mrs. Maisel,"Drama, Comedy",3.0,TV-MA,Yes
3,Four More Shots Please,"Drama, Comedy",2.0,TV-MA,Yes
4,Fleabag,Comedy,2.0,TV-MA,Yes


In [18]:
# Counted Each TV Show in each Dataframe to find the Largest Streaming Service
Total_Hulu = dataCleaned.shape[0]
Total_Netflix = final_netflix_data.shape[0]
Total_Prime = Cleaned_Prime_Data.shape[0]

if Total_Hulu > Total_Netflix and Total_Hulu > Total_Prime:
    Service_With_The_Most = "Hulu"
    Most_Info = Total_Hulu
else:
    Hulu_Service = "Hulu"

if Total_Netflix > Total_Hulu and Total_Netflix > Total_Prime:
    Service_With_The_Most = "Netflix"
    Most_Info = Total_Netflix
else:
    Netflix_Service = "Netflix"

if Total_Prime > Total_Hulu and Total_Prime > Total_Netflix:
    Service_With_The_Most = "Prime"
    Most_Info = Total_Prime
else:
    Prime_Service = "Prime"

print (f"{Service_With_The_Most} has {Most_Info} TV Shows.")
print (f"{Hulu_Service} has {Total_Hulu} TV Shows.")
print (f"{Prime_Service} has {Total_Prime} TV Shows.")
print (f"{Service_With_The_Most} has the most available information with {Most_Info} TV Shows data.")


Netflix has 1967 TV Shows.
Hulu has 1000 TV Shows.
Prime has 393 TV Shows.
Netflix has the most available information with 1967 TV Shows data.
